In [60]:
import pandas as pd

x_train = pd.read_csv('bike_x_train.csv', encoding = 'cp949')
x_test = pd.read_csv('bike_x_test.csv', encoding = 'cp949')
y_train = pd.read_csv('bike_y_train.csv', encoding = 'cp949')

print(y_train.head())

          datetime  count
0  2011-01-01 0:00     16
1  2011-01-01 1:00     40
2  2011-01-01 2:00     32
3  2011-01-01 3:00     13
4  2011-01-01 4:00      1


In [61]:
y_train = y_train['count']

In [5]:
print(x_train.head())

          datetime  계절  공휴일  근무일  날씨    온도    체감온도  습도   풍속
0  2011-01-01 0:00   1    0    0   1  9.84  14.395  81  0.0
1  2011-01-01 1:00   1    0    0   1  9.02  13.635  80  0.0
2  2011-01-01 2:00   1    0    0   1  9.02  13.635  80  0.0
3  2011-01-01 3:00   1    0    0   1  9.84  14.395  75  0.0
4  2011-01-01 4:00   1    0    0   1  9.84  14.395  75  0.0


In [4]:
print(x_train.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10886 entries, 0 to 10885
Data columns (total 9 columns):
datetime    10886 non-null object
계절          10886 non-null int64
공휴일         10886 non-null int64
근무일         10886 non-null int64
날씨          10886 non-null int64
온도          10886 non-null float64
체감온도        10886 non-null float64
습도          10886 non-null int64
풍속          10886 non-null float64
dtypes: float64(3), int64(5), object(1)
memory usage: 765.5+ KB
None


In [62]:
def date(x):
    x['datetime'] = pd.to_datetime(x['datetime'])
    x['year'] = x['datetime'].dt.year
    x['month'] = x['datetime'].dt.month
    x['day'] = x['datetime'].dt.day
    x['hour'] = x['datetime'].dt.hour
    x = x.drop(columns='datetime')
    return x

In [63]:
x_train1 = date(x_train)
x_test1 = date(x_test)
# y_train = date(y_train)

In [64]:
x_train1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10886 entries, 0 to 10885
Data columns (total 12 columns):
계절       10886 non-null int64
공휴일      10886 non-null int64
근무일      10886 non-null int64
날씨       10886 non-null int64
온도       10886 non-null float64
체감온도     10886 non-null float64
습도       10886 non-null int64
풍속       10886 non-null float64
year     10886 non-null int64
month    10886 non-null int64
day      10886 non-null int64
hour     10886 non-null int64
dtypes: float64(3), int64(9)
memory usage: 1020.6 KB


In [65]:
x_train1.corr() ## 강한 상관관계(>= 0.6) : 계절-month, 온도-체감온도

,계절,공휴일,근무일,날씨,온도,체감온도,습도,풍속,year,month,day,hour
계절,1.000000,0.029368,-0.008126,0.008879,0.258689,0.264744,0.190610,-0.147121,-0.004797,0.971524,0.001729,-0.006546
공휴일,0.029368,1.000000,-0.250491,-0.007074,0.000295,-0.005215,0.001929,0.008409,0.012021,0.001731,-0.015877,-0.000354
근무일,-0.008126,-0.250491,1.000000,0.033772,0.029966,0.024660,-0.010880,0.013373,-0.002482,-0.003394,0.009829,0.002780
날씨,0.008879,-0.007074,0.033772,1.000000,-0.055035,-0.055376,0.406244,0.007261,-0.012548,0.012144,-0.007890,-0.022740
온도,0.258689,0.000295,0.029966,-0.055035,1.000000,0.984948,-0.064949,-0.017852,0.061226,0.257589,0.015551,0.145430
체감온도,0.264744,-0.005215,0.024660,-0.055376,0.984948,1.000000,-0.043536,-0.057473,0.058540,0.264173,0.011866,0.140343
습도,0.190610,0.001929,-0.010880,0.406244,-0.064949,-0.043536,1.000000,-0.318607,-0.078606,0.204537,-0.011335,-0.278011
풍속,-0.147121,0.008409,0.013373,0.007261,-0.017852,-0.057473,-0.318607,1.000000,-0.015221,-0.150192,0.036157,0.146631
year,-0.004797,0.012021,-0.002482,-0.012548,0.061226,0.058540,-0.078606,-0.015221,1.000000,-0.004932,0.001800,-0.004234
month,0.971524,0.001731,-0.003394,0.012144,0.257589,0.264173,0.204537,-0.150192,-0.004932,1.000000,0.001974,-0.006818


In [66]:
# 다중공선성 제거
x_train2 = x_train1.drop(columns=['month', '체감온도'])
x_test2 = x_test1.drop(columns=['month', '체감온도'])

In [67]:
from sklearn.model_selection import train_test_split

x_tr, x_val, y_tr, y_val = train_test_split(x_train2, y_train, test_size = 0.2, random_state = 123)

In [68]:
## 검증셋 예측&평가
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score

rf = RandomForestRegressor(random_state = 12)
rf.fit(x_tr, y_tr)
pred_v = rf.predict(x_val)
score = r2_score(y_val, pred_v)

print(score)

0.9342735026650105


In [70]:
## 원본데이터 예측
rf = RandomForestRegressor(random_state = 12)
rf.fit(x_train2, y_train)
pred = rf.predict(x_test2)

In [71]:
## 제출
sub = pd.DataFrame({'datetime' : x_test['datetime'], 'count' : pred})
sub.to_csv('00012314.csv', index = False)

In [73]:
## 제출 확인
check = pd.read_csv('00012314.csv')
print(check.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6493 entries, 0 to 6492
Data columns (total 2 columns):
datetime    6493 non-null object
count       6493 non-null float64
dtypes: float64(1), object(1)
memory usage: 101.5+ KB
None
